In [6]:
from sklearn.preprocessing import StandardScaler
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import mysql.connector
import pandas as pd

import json

with open('config.json') as f:
    config = json.load(f)


cnx = mysql.connector.connect(user=config['user'],
                              password=config['password'],
                              host=config['host'],
                              database=config['database'])

cursor = cnx.cursor()

# Example query
query = ("SELECT * FROM PH_Temp_Diet")

cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()

# Create a pandas DataFrame
data = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])


cursor.close()
cnx.close()

In [4]:
df3 =data
df3 = df3.drop(['id'], axis=1)

# Pivot the data to create columns for each unique food_name
pivoted_data = df3.pivot_table(index=['person_id', 'date','PH','temperature'], columns='food_name', values='amount')
pivoted_data
# Rename columns to remove the 'food_name' label prefix
pivoted_data.columns.name = None
pivoted_data.columns = pivoted_data.columns.str.replace('food_name_', '')

# # Join the pivoted data with the original data on the 'person_id' and 'date' columns
joined_data = pd.merge(data.drop(columns=['food_name', 'amount','id']), pivoted_data, on=['person_id', 'date','temperature','PH'])

# # Save the joined data to a new CSV file
#df3 = pivoted_data
# Group by 'person_id' and 'date', and aggregate the other columns
grouped_df = joined_data.groupby(['person_id', 'date','temperature','PH'], as_index=False);
joined_data = joined_data.drop_duplicates()
joined_data

,PH,temperature,date,person_id,Apple,Apple Pie,Banana,Beer,Braised Potatoes,Bread,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,5.40,35.8,06.09.2017,1,0.0,0.0,100.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,150.0
97,6.03,36.1,07.09.2017,1,0.0,0.0,0.0,0.0,0.0,180.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,50.0
190,5.81,36.4,08.09.2017,1,0.0,0.0,0.0,0.0,0.0,250.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,50.0
283,6.10,36.6,09.09.2017,1,0.0,0.0,110.0,500.0,0.0,100.0,...,0.0,0.0,0.0,60.0,0.0,0.0,400.0,0.0,0.0,0.0
376,6.06,35.9,10.09.2017,1,0.0,0.0,100.0,0.0,0.0,150.0,...,0.0,0.0,0.0,30.0,0.0,0.0,300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19441,5.82,36.5,03.04.2018,1,140.0,0.0,78.0,0.0,0.0,43.0,...,0.0,0.0,0.0,0.0,0.0,65.0,1050.0,0.0,0.0,0.0
19534,5.03,36.1,04.04.2018,1,214.0,0.0,0.0,0.0,0.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0
19627,5.59,36.3,05.04.2018,1,0.0,0.0,0.0,0.0,0.0,190.0,...,0.0,250.0,0.0,34.0,0.0,0.0,1350.0,0.0,0.0,0.0
19720,5.40,35.8,06.09.2017,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Define a function to recommend food items based on pH level
def recommend_foods(ph_level, df):
    # Calculate the mean pH for each food item
    food_phs = df.iloc[:, 5:].mean()
    # Determine which food items are associated with low or high pH levels
    low_phs = food_phs[food_phs < ph_level].index.tolist()
    high_phs = food_phs[food_phs > ph_level].index.tolist()
    
    # Return the recommended food items
    if ph_level < 7:
        return high_phs
    else:
        return low_phs

In [10]:
recommend_foods(6.8,joined_data)

['Banana',
 'Beer',
 'Braised Potatoes',
 'Bread',
 'Cake',
 'Cheese',
 'Cookie',
 'Croissant',
 'Curd',
 'Curd Pie',
 'Dried apricots',
 'Fried potatoes',
 'Fruit tea',
 'Ice cream',
 'Latte',
 'Lemon water',
 'Liquid',
 'Mashed potatoes',
 'Mayonnaise salad',
 'Meat',
 'Meat pilaf',
 'Meatless pilaf',
 'Melon',
 'Mineral water (Borjomi)',
 'Mineral water (Esentuki - 4)',
 'Mineral water (Prolom)',
 'Morse',
 'Noodles with shrimp',
 'Oatmeal',
 'Orange juice',
 'Patty with cabbage',
 'Peach',
 'Pear',
 'Pizza with meat',
 'Plum',
 'Potato dumplings',
 'Pumpkin seeds',
 'Red wine',
 'Scrambled eggs',
 'Sushi',
 'Tangerines',
 'Tea(black/green)',
 'The nuts',
 'Vegetable salad',
 'Water',
 'Watermelon',
 'White wine',
 'Zucchini Fritters']

In [37]:
df2= data
df2.head(5)

,id,PH,temperature,food_name,amount,date,person_id
0,39446,5.4,35.8,Gym,0.0,06.09.2017,1
1,39447,5.4,35.8,Liquid,1300.0,06.09.2017,1
2,39448,5.4,35.8,Lemon water,250.0,06.09.2017,1
3,39449,5.4,35.8,Tea(black/green),0.0,06.09.2017,1
4,39450,5.4,35.8,Fruit tea,600.0,06.09.2017,1


In [3]:
import pandas as pd

# Load data from your database or CSV file
df3 = data

df3 = df3.drop(['id'], axis=1)

# Pivot the data to create columns for each unique food_name
pivoted_data = df3.pivot_table(index=['person_id', 'date','PH','temperature'], columns='food_name', values='amount')
pivoted_data
# Rename columns to remove the 'food_name' label prefix
pivoted_data.columns.name = None
pivoted_data.columns = pivoted_data.columns.str.replace('food_name_', '')

# # Join the pivoted data with the original data on the 'person_id' and 'date' columns
joined_data = pd.merge(data.drop(columns=['food_name', 'amount','id']), pivoted_data, on=['person_id', 'date','temperature','PH'])

# # Save the joined data to a new CSV file
#df3 = pivoted_data
# Group by 'person_id' and 'date', and aggregate the other columns
grouped_df = joined_data.groupby(['person_id', 'date','temperature','PH'], as_index=False);
joined_data = joined_data.drop_duplicates()
joined_data

,PH,temperature,date,person_id,Apple,Apple Pie,Banana,Beer,Braised Potatoes,Bread,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,5.40,35.8,06.09.2017,1,0.0,0.0,100.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,150.0
97,6.03,36.1,07.09.2017,1,0.0,0.0,0.0,0.0,0.0,180.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,50.0
190,5.81,36.4,08.09.2017,1,0.0,0.0,0.0,0.0,0.0,250.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,50.0
283,6.10,36.6,09.09.2017,1,0.0,0.0,110.0,500.0,0.0,100.0,...,0.0,0.0,0.0,60.0,0.0,0.0,400.0,0.0,0.0,0.0
376,6.06,35.9,10.09.2017,1,0.0,0.0,100.0,0.0,0.0,150.0,...,0.0,0.0,0.0,30.0,0.0,0.0,300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19441,5.82,36.5,03.04.2018,1,140.0,0.0,78.0,0.0,0.0,43.0,...,0.0,0.0,0.0,0.0,0.0,65.0,1050.0,0.0,0.0,0.0
19534,5.03,36.1,04.04.2018,1,214.0,0.0,0.0,0.0,0.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0
19627,5.59,36.3,05.04.2018,1,0.0,0.0,0.0,0.0,0.0,190.0,...,0.0,250.0,0.0,34.0,0.0,0.0,1350.0,0.0,0.0,0.0
19720,5.40,35.8,06.09.2017,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from sklearn.model_selection import train_test_split

X= joined_data.iloc[:,0:2]
X

,PH,temperature
0,5.40,35.8
97,6.03,36.1
190,5.81,36.4
283,6.10,36.6
376,6.06,35.9
...,...,...
19441,5.82,36.5
19534,5.03,36.1
19627,5.59,36.3
19720,5.40,35.8


In [5]:
y = joined_data.iloc[:,5:]
y

,Apple Pie,Banana,Beer,Braised Potatoes,Bread,Buckwheat porridge,Bun,Cake,Candies,Cheese,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,150.0
97,0.0,0.0,0.0,0.0,180.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,50.0
190,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,50.0
283,0.0,110.0,500.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,60.0,0.0,0.0,400.0,0.0,0.0,0.0
376,0.0,100.0,0.0,0.0,150.0,0.0,150.0,0.0,0.0,0.0,...,0.0,0.0,0.0,30.0,0.0,0.0,300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19441,0.0,78.0,0.0,0.0,43.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,65.0,1050.0,0.0,0.0,0.0
19534,0.0,0.0,0.0,0.0,150.0,0.0,0.0,69.0,0.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0
19627,0.0,0.0,0.0,0.0,190.0,0.0,0.0,55.0,0.0,29.0,...,0.0,250.0,0.0,34.0,0.0,0.0,1350.0,0.0,0.0,0.0
19720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
y = data.groupby(['person_id', 'date'])[['food_name', 'amount']].agg(list)
# Reset the index
y = y.reset_index()

# Custom function to combine food_name and amount into one list
def combine_food_and_amount(food_names, amounts):
    combined = []
    for food, amount in zip(food_names, amounts):
        combined.append((food, amount))
    return combined

# Apply the custom function to combine food_name and amount columns
y['food_and_amount'] = y.apply(lambda row: combine_food_and_amount(row['food_name'], row['amount']), axis=1)

# Drop the food_name and amount columns
y = y.drop(['food_name', 'amount','person_id','date'], axis=1)

# Print the modified DataFrame
print(y.head(5))

                                     food_and_amount
0  [(Gym, 0.0), (Liquid, 1250.0), (Lemon water, 2...
1  [(Gym, 0.0), (Liquid, 2500.0), (Lemon water, 3...
2  [(Gym, 1.0), (Liquid, 3050.0), (Lemon water, 3...
3  [(Gym, 1.0), (Liquid, 2900.0), (Lemon water, 3...
4  [(Gym, 1.0), (Liquid, 1750.0), (Lemon water, 2...


In [14]:
x.shape

NameError: name 'x' is not defined

In [107]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.6, random_state=42)
y_train

,food_and_amount
64,"[(Gym, 0.0), (Liquid, 2920.0), (Lemon water, 3..."
44,"[(Gym, 0.0), (Liquid, 2825.0), (Lemon water, 3..."
139,"[(Gym, 0.0), (Liquid, 1250.0), (Lemon water, 2..."
40,"[(Gym, 1.0), (Liquid, 1400.0), (Lemon water, 2..."
123,"[(Gym, 0.0), (Liquid, 1550.0), (Lemon water, 2..."
...,...
106,"[(Gym, 0.0), (Liquid, 3450.0), (Lemon water, 3..."
14,"[(Gym, 0.0), (Liquid, 1250.0), (Lemon water, 2..."
92,"[(Gym, 0.0), (Liquid, 3275.0), (Lemon water, 3..."
179,"[(Gym, 1.0), (Liquid, 1400.0), (Lemon water, 2..."


In [108]:
from sklearn.linear_model import LinearRegression
# Train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = lr.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# print("MSE:", mse)

ValueError: setting an array element with a sequence.

In [ ]:
import pickle
with open('recommend_diet.pkl', 'wb') as f:
    pickle.dump(joined_data, f)